In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [2]:
# program = "graph_coloring"
program = "maximal_matching"
# program = "dijkstra_token_ring"
# graph_name = "graph_7"
graph_name = "graph_10"

no_simulations = 10000
fault_interval = 2
sched = 0

selected_nodes = [0, 4, 7, 9]

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__duong"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,0,30.692981
1,1,30.600387
2,2,30.508071
3,3,30.416034
4,4,30.324274
...,...,...
5249,5249,-0.037907
5250,5250,-0.037907
5251,5251,-0.037907
5252,5252,-0.037907


In [5]:
dfs[0]

,Steps,exp_fit
0,0,31.227303
1,1,31.130787
2,2,31.034569
3,3,30.938649
4,4,30.843024
...,...,...
4149,4149,-0.029418
4150,4150,-0.029418
4151,4151,-0.029419
4152,4152,-0.029419


In [6]:
dfs[1]

,Steps,exp_fit
0,0,30.999441
1,1,30.904852
2,2,30.810552
3,3,30.716538
4,4,30.622812
...,...,...
4125,4125,-0.045774
4126,4126,-0.045774
4127,4127,-0.045774
4128,4128,-0.045775


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__4,exp_fit__7,exp_fit__9
Steps,,,,
0,31.227303,30.999441,30.692981,26.654159
1,31.130787,30.904852,30.600387,26.582611
2,31.034569,30.810552,30.508071,26.511255
3,30.938649,30.716538,30.416034,26.440091
4,30.843024,30.622812,30.324274,26.369118
...,...,...,...,...
4125,-0.029412,-0.045774,-0.037791,0.018326
4126,-0.029412,-0.045774,-0.037791,0.018325
4127,-0.029412,-0.045774,-0.037791,0.018324


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 4,Exponential fit of node 7,Exponential fit of node 9
Steps,,,,
0,31.227303,30.999441,30.692981,26.654159
1,31.130787,30.904852,30.600387,26.582611
2,31.034569,30.810552,30.508071,26.511255
3,30.938649,30.716538,30.416034,26.440091
4,30.843024,30.622812,30.324274,26.369118
...,...,...,...,...
4125,-0.029412,-0.045774,-0.037791,0.018326
4126,-0.029412,-0.045774,-0.037791,0.018325
4127,-0.029412,-0.045774,-0.037791,0.018324


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [11]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}__{fault_interval}.png",
    bbox_inches="tight",
)
plt.close()